Make API call of Guardian to return all Jay Rayner articles. First query the API for the list of articles - this returns a page of results. We fetch the first batch of links to follow here, then we need to work through the rest of the list of pages, getting all those links.  

In [15]:
import requests
import re 
import os

page_size = 50
page = 1
api_key = os.environ.get('GUARDIAN_API_KEY')

#base url which looks for the jay tags 
url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}&page={page}"
payload = {}
headers = {
  'Cookie': 'AWSELB=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; AWSELBCORS=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; BCSI-CS-682e9ee7ab0fdcff=1'
}

#function to go through the results
def review_grabber(apiresults):
    with open('jaymap.csv', 'a') as csvfile:
        #if status is ok, work through the list of results and find the postcodes in the linked articles
        if apiresults['response']['status'] == 'ok':
            count = 1
            for result in apiresults['response']['results']:
                article = requests.request("GET", result['webUrl'], headers=headers, data = payload)
                postcode = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', article.text)
                if len(postcode) > 0:
                    csvfile.write(f"\n{count},{result['webUrl']},{postcode[0]}")
                    print(count, result['webUrl'], postcode[0])
                count += 1

#fetch the first page listing articles by jay
response = requests.request("GET", url, headers=headers, data = payload)
responsejson = response.json()
#how many pages are there in the response, as we will need to get them all
pages = responsejson['response']['pages']
#total articles to check off later
total_articles = responsejson['response']['total']
print(f"there are {pages} pages of articles to get, {page_size} articles on each page, {total_articles} articles in total")
#send the whole response into the function to visit the page and get the postcode
review_grabber(responsejson)

there are 22 pages of articles to get, 50 articles on each page, 1073 articles in total
1 https://www.theguardian.com/food/2021/aug/29/jay-rayner-restaurant-review-the-gpo-liverpool-could-be-first-class-when-its-sorted L1 6DA
2 https://www.theguardian.com/food/2021/aug/22/kurisu-omakase-at-ichiban-sushi-london-it-is-both-dinner-and-theatre-restaurant-review SW9 8PY
3 https://www.theguardian.com/food/2021/aug/15/the-polo-lounge-at-the-dorchester-hotel-dismal-food-at-eye-popping-prices-restaurant-review W1K 1QA
4 https://www.theguardian.com/food/2021/aug/08/jay-rayner-restaurant-review-burnt-orange-in-brighton-delivers-on-its-smoky-promise BN1 1AL
5 https://www.theguardian.com/food/2021/aug/01/jay-rayner-restaurant-review-the-bradley-hare-wiltshire-the-meat-is-impeccable BA12 7HW
6 https://www.theguardian.com/food/2021/jul/25/jay-rayner-restaurant-review-sonny-stores-bristol-they-get-things-absolutely-right BS3 1QS
7 https://www.theguardian.com/food/2021/jul/18/jay-rayner-restaurant-revi

In [16]:
#work through the rest of the pages getting the links
while page <= pages:
    page += 1
    print(page)
    url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}&page={page}"
    response2 = requests.request("GET", url, headers=headers, data = payload)
    response2json = response2.json()
    review_grabber(response2json)

2
1 https://www.theguardian.com/food/2020/aug/23/jay-rayner-restaurant-review-lahpet-london-visit-for-superb-burmese-food E1 6JW
2 https://www.theguardian.com/food/2020/aug/16/turnips-london-a-clever-response-to-challenging-times-restaurant-review SE1 9AH
3 https://www.theguardian.com/food/2020/aug/09/jay-rayner-restaurant-review-kolamba-london-a-lovely-dinner-in-a-calming-space W1B 5QA
4 https://www.theguardian.com/food/2020/aug/02/street-urchin-manchester-a-place-where-good-things-happen-restaurant-review M4 5BG
5 https://www.theguardian.com/food/2020/jul/26/waterside-bistro-shipley-a-much-missed-old-friend-jay-rayner-restaurant-review BD17 7DW
6 https://www.theguardian.com/food/2020/jul/19/at-last-going-out-for-lunch-at-picque-nique-a-french-place-really-worth-the-wait SE1 3LD
21 https://www.theguardian.com/food/2020/apr/05/flying-frenchman-la-canteen-london-order-his-food-if-you-live-nearby-restaurant-review N16 9PS
23 https://www.theguardian.com/food/2020/mar/22/farmyard-norwich-f